# Загрузка данных в Qdrant

Этот ноутбук:
1. Удаляет все существующие документы из Qdrant
2. Загружает данные из JSON файла с сообщениями
3. Добавляет документы в векторную базу данных
4. Проверяет корректность загрузки


## 1. Импорты и настройка


In [ ]:
import json
import logging
import sys
from datetime import datetime
from pathlib import Path

# Добавляем путь к проекту
project_root = Path("/srv/nlp1/eval_dir/T-bank_NLP_")
sys.path.insert(0, str(project_root))

# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

print("=" * 80)
print(f"📦 ЗАГРУЗКА ДАННЫХ В QDRANT - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

📦 ЗАГРУЗКА ДАННЫХ В QDRANT - 2025-11-18 08:49:46


## 2. Инициализация RetrieverService


In [1]:
from tplexity.retriever.retriever_service import RetrieverService

print("🔄 Инициализация RetrieverService...")
service = RetrieverService()
print("✅ RetrieverService инициализирован")

SyntaxError: invalid syntax (config.py, line 21)

2025-11-17 21:32:36,131 - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents "HTTP/1.1 200 OK"



🔍 Проверка конфигурации коллекции...
  ✅ Dense векторы: ❌ НЕТ
  ✅ Sparse векторы (BM25): Да

⚠️  ВНИМАНИЕ: Коллекция не настроена правильно!
   Нужно пересоздать коллекцию с dense и sparse векторами.


## 3. Проверка количества документов (до очистки)


In [ ]:
count_before = await service.vector_search.client.count(collection_name=service.vector_search.collection_name)
print(f"📊 Документов в коллекции '{service.vector_search.collection_name}' (до очистки): {count_before.count}")

2025-11-17 21:32:36,186 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/count "HTTP/1.1 200 OK"


📊 Документов в коллекции 'documents' (до очистки): 1000


## 4. Удаление всех документов


In [ ]:
print("⚠️  Удаление всех документов из Qdrant...")
await service.delete_all_documents()
print("✅ Все документы удалены")

# Проверяем, что коллекция пуста
count_after_delete = await service.vector_search.client.count(collection_name=service.vector_search.collection_name)
print(f"📊 Документов после очистки: {count_after_delete.count}")

2025-11-17 21:32:36,194 - WARNING - ⚠️ [retriever_service] Удаление всех документов
2025-11-17 21:32:36,195 - WARNING - ⚠️ [retriever][vector_search] Удаление всех документов из коллекции
2025-11-17 21:32:36,266 - INFO - HTTP Request: DELETE https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents "HTTP/1.1 200 OK"
2025-11-17 21:32:36,268 - INFO - ✅ [retriever][vector_search] Коллекция documents удалена
2025-11-17 21:32:36,312 - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"


⚠️  Удаление всех документов из Qdrant...


2025-11-17 21:32:36,556 - INFO - HTTP Request: PUT https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents "HTTP/1.1 200 OK"
2025-11-17 21:32:36,557 - INFO - ✅ [retriever][vector_search] Коллекция documents создана с dense и sparse векторами
2025-11-17 21:32:36,558 - INFO - ✅ [retriever][vector_search] Коллекция documents пересоздана
2025-11-17 21:32:36,558 - INFO - ✅ [retriever_service] Все документы удалены
2025-11-17 21:32:36,603 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/count "HTTP/1.1 200 OK"


✅ Все документы удалены
📊 Документов после очистки: 0


## 5. Загрузка данных из JSON


In [ ]:
# Путь к JSON файлу с сообщениями
messages_path = project_root / "src/eval/eval_data/messages_diverse_1000posts_all_channels.json"

print(f"🔄 Загрузка сообщений из {messages_path.name}...")
with open(messages_path, encoding="utf-8") as f:
    messages = json.load(f)

print(f"✅ Загружено {len(messages)} сообщений из файла")
print("\n📋 Пример первого сообщения:")
print(f"  ID: {messages[0]['id']}")
print(f"  Channel ID: {messages[0]['channel_id']}")
print(f"  Date: {messages[0].get('date', 'N/A')}")
print(f"  Text: {messages[0].get('text', '')[:100]}...")

🔄 Загрузка сообщений из messages_diverse_1000posts_all_channels.json...
✅ Загружено 1000 сообщений из файла

📋 Пример первого сообщения:
  ID: 7078
  Channel ID: 1418181070
  Date: 2025-11-14T12:14:02+00:00
  Text: **Вместе мы сильнее: запустили сообщество для инвесторов ****❤️**

Пока оно доступно в обновлённом п...


## 6. Подготовка данных для индексации


In [ ]:
print("🔄 Подготовка документов для индексации...")

documents = []
metadatas = []

for msg in messages:
    text = msg.get("text", "").strip()
    if not text:
        continue

    documents.append(text)

    # Метаданные (составной ID для связи с ground truth)
    metadata = {
        "doc_id": f"{msg['channel_id']}_{msg['id']}",  # Составной ID
        "message_id": msg["id"],
        "channel_id": msg["channel_id"],
        "date": msg.get("date", ""),
        "link": msg.get("link", ""),
    }
    metadatas.append(metadata)

print(f"✅ Подготовлено {len(documents)} документов для индексации")
print(f"📊 Пропущено {len(messages) - len(documents)} пустых сообщений")

🔄 Подготовка документов для индексации...
✅ Подготовлено 1000 документов для индексации
📊 Пропущено 0 пустых сообщений


## 7. Добавление документов в Qdrant


In [ ]:
print("🔄 Загрузка документов в Qdrant...")
print("⏳ Это может занять несколько минут...")

# Добавляем документы (UUID будут сгенерированы автоматически)
await service.add_documents(documents=documents, metadatas=metadatas)

print("✅ Документы успешно добавлены в Qdrant")

2025-11-17 21:32:36,647 - INFO - 🔄 [retriever_service] Добавление 1000 новых документов


🔄 Загрузка документов в Qdrant...
⏳ Это может занять несколько минут...


Batches: 100%|██████████| 32/32 [00:49<00:00,  1.55s/it]
2025-11-17 21:33:26,980 - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"
2025-11-17 21:33:27,028 - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents "HTTP/1.1 200 OK"
2025-11-17 21:33:27,029 - INFO - ✅ [retriever][vector_search] Коллекция documents уже существует с правильной конфигурацией
2025-11-17 21:33:27,303 - INFO - HTTP Request: PUT https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"
2025-11-17 21:33:27,305 - INFO - 📦 [retriever][vector_search] Загружено 10/1000 документов
2025-11-17 21:33:27,473 - INFO - HTTP Request: PUT https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points?wait=true "HTTP/1.1 200 OK"


✅ Документы успешно добавлены в Qdrant


## 8. Проверка количества документов (после загрузки)


In [ ]:
count_after_load = await service.vector_search.client.count(collection_name=service.vector_search.collection_name)

print("=" * 80)
print("📊 ИТОГОВАЯ СТАТИСТИКА")
print("=" * 80)
print(f"📈 Документов до очистки:     {count_before.count}")
print(f"🗑️  Документов после очистки:  {count_after_delete.count}")
print(f"✅ Документов после загрузки:  {count_after_load.count}")
print(f"📦 Добавлено документов:       {len(documents)}")
print("=" * 80)

if count_after_load.count == len(documents):
    print("✅ Все документы успешно загружены!")
else:
    print(f"⚠️  Расхождение: ожидалось {len(documents)}, загружено {count_after_load.count}")

2025-11-17 21:33:49,831 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/count "HTTP/1.1 200 OK"


📊 ИТОГОВАЯ СТАТИСТИКА
📈 Документов до очистки:     1000
🗑️  Документов после очистки:  0
✅ Документов после загрузки:  1000
📦 Добавлено документов:       1000
✅ Все документы успешно загружены!


## 9. Тестовый поиск


In [ ]:
print("\n🔍 Тестовый поиск для проверки работоспособности...\n")

test_query = "инвестиции акции"
results = await service.search(test_query, top_k=5, top_n=3, use_rerank=False)

print(f"Запрос: '{test_query}'")
print(f"Найдено результатов: {len(results)}\n")

for i, (doc_id, score, text, metadata) in enumerate(results, 1):
    print(f"Результат {i}:")
    print(f"  Doc ID: {doc_id}")
    print(f"  Score: {score:.3f}")
    print(f"  Text: {text[:150]}...")
    print(f"  Metadata doc_id: {metadata.get('doc_id', 'N/A')}")
    print(f"  Link: {metadata.get('link', 'N/A')}")
    print()

2025-11-17 21:33:53,782 - INFO - 🔍 [retriever_service] Начало поиска для запроса: инвестиции акции...
2025-11-17 21:33:53,783 - INFO - 🔄 [retriever_service.search] Выполнение hybrid поиска (Dense + BM25 + RRF), top_k: 5
2025-11-17 21:33:53,783 - INFO - 🔄 [retriever_service.search] Вызываем vector_search.search с: query=инвестиции акции, top_k=5, search_type='hybrid'



🔍 Тестовый поиск для проверки работоспособности...



Batches: 100%|██████████| 1/1 [00:00<00:00, 50.96it/s]
2025-11-17 21:33:53,899 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-17 21:33:53,901 - INFO - ✅ [retriever_service] Hybrid поиск завершен, найдено результатов: 5
2025-11-17 21:33:53,901 - INFO - ✅ [retriever_service] Поиск завершен, возвращено 3 результатов


Запрос: 'инвестиции акции'
Найдено результатов: 3

Результат 1:
  Doc ID: 6c686a4a-5b1f-4ec1-bb84-2d015b6d4ced
  Score: 0.500
  Text: **Знаете игру «Алхимию», где нужно смешивать простые элементы, создавая новое? Альфа-Инвестиции сыграли в неё с инвесторами — и выиграли все ****🎆**

...
  Metadata doc_id: 1418181070_6696
  Link: https://t.me/alfa_investments/6696

Результат 2:
  Doc ID: 518e9728-3cc8-4274-92ed-40952db1be22
  Score: 0.500
  Text: **Альфа-Инвестиции + Промомед = рекордная сделка с облигациями нового формата ****😍**

Альфа-Инвестиции снова подтвердили статус лидера альтернативных...
  Metadata doc_id: 1418181070_6587
  Link: https://t.me/alfa_investments/6587

Результат 3:
  Doc ID: be3f76e2-3d7a-420c-84f4-e03b91efac6b
  Score: 0.333
  Text: **Школьные линейки позади, пришло время биржевых. Три. Два. Один. Новый учебный год инвесторов объявляется открытым ****🔔**

Загляните в наше приложен...
  Metadata doc_id: 1418181070_6702
  Link: https://t.me/alfa_investments/6702



## 10. Проверка конкретного документа по ground truth ID


In [ ]:
# Проверяем, что можем найти конкретный документ
print("\n🔍 Проверка поиска конкретного документа...\n")

# Берем первое сообщение из нашего датасета
first_msg = messages[0]
expected_doc_id = f"{first_msg['channel_id']}_{first_msg['id']}"

print(f"Ожидаемый doc_id: {expected_doc_id}")
print(f"Текст документа: {first_msg.get('text', '')[:150]}...\n")

# Ищем по тексту из этого документа
search_text = first_msg.get("text", "")[:100]  # Берем первые 100 символов
results = await service.search(search_text, top_k=5, top_n=5, use_rerank=False)

print("Результаты поиска (топ-5):")
found = False
for i, (doc_id, score, text, metadata) in enumerate(results, 1):
    metadata_doc_id = metadata.get("doc_id", "")
    is_target = metadata_doc_id == expected_doc_id
    marker = "✅ [ЦЕЛЕВОЙ ДОКУМЕНТ]" if is_target else ""

    print(f"  {i}. doc_id: {metadata_doc_id}, Score: {score:.3f} {marker}")

    if is_target:
        found = True

print()
if found:
    print("✅ Целевой документ найден! Система работает корректно.")
else:
    print("⚠️  Целевой документ не найден в топ-5. Это может быть нормально.")

2025-11-17 21:33:44,266 - INFO - 🔍 [retriever_service] Начало поиска для запроса: **Вместе мы сильнее: запустили сообщество для инве...
2025-11-17 21:33:44,267 - INFO - 🔄 [retriever_service.search] Выполнение hybrid поиска (Dense + BM25 + RRF), top_k: 5
2025-11-17 21:33:44,268 - INFO - 🔄 [retriever_service.search] Вызываем vector_search.search с: query=**Вместе мы сильнее: запустили сообщество для инве, top_k=5, search_type='hybrid'



🔍 Проверка поиска конкретного документа...

Ожидаемый doc_id: 1418181070_7078
Текст документа: **Вместе мы сильнее: запустили сообщество для инвесторов ****❤️**

Пока оно доступно в обновлённом приложении Альфа-Инвестиций для андроида, но скоро ...



Batches: 100%|██████████| 1/1 [00:00<00:00, 48.63it/s]
2025-11-17 21:33:44,346 - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-17 21:33:44,347 - INFO - ✅ [retriever_service] Hybrid поиск завершен, найдено результатов: 5
2025-11-17 21:33:44,348 - INFO - ✅ [retriever_service] Поиск завершен, возвращено 5 результатов


Результаты поиска (топ-5):
  1. doc_id: 1418181070_7078, Score: 1.000 ✅ [ЦЕЛЕВОЙ ДОКУМЕНТ]
  2. doc_id: 1239405989_10908, Score: 0.333 
  3. doc_id: 1498653424_21593, Score: 0.333 
  4. doc_id: 1351339368_8827, Score: 0.250 
  5. doc_id: 1344086554_12388, Score: 0.250 

✅ Целевой документ найден! Система работает корректно.


## Готово! 🎉

Данные успешно загружены в Qdrant. Теперь можно запускать evaluation ноутбуки для оценки качества поиска.
